In [ ]:
!pip install beautifulsoup4
!pip install -U textblob
!pip install nltk

In [ ]:
from bs4 import BeautifulSoup
import os,shutil,requests,csv
import pandas as pd
import numpy as np
import re
import textblob
import nltk
from textblob import TextBlob
nltk.download('stopwords')

In [ ]:
def DownloadArticle():
    URL = r'http://www.thehindu.com/archive/print/2017/11/{}/'
    BOLD = '\033[1m';END = '\033[0m'
    DATES = [15,16,17,18,19]
    CITIES = ['New Delhi','Mumbai','Bengaluru','Chennai',
              'Hyderabad','Kolkata','Kochi','Vijayawada','Visakhapatnam']
    header = ['cityname','Article']
    print('data will be scrapped from 15th to 19th november')
    with open('Articles.csv','w',encoding='utf-8') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow(header)
        for x in DATES:
            print(BOLD+'getting articles for '+END + URL.format(x))
            resp = requests.get(URL.format(x))
            soup = BeautifulSoup(resp.content, 'lxml')
            for d in CITIES:
                count = 0
                print(BOLD + d + END,end='') #city
                for c in soup.find_all('h4',text=d.lower()):
                #print(c.text)

                    nex = c.find_next_sibling()
                    for UR in nex.find_all('a'):
                        count+=1
                        article = []
                        ARTICLEURL = UR.get('href')
                        #print(ARTICLEURL)
                        res2 = requests.get(ARTICLEURL)
                        soup2 = BeautifulSoup(res2.content,'lxml')
                        for _ in soup2.find_all(['p'],_class=''):
                            article.append(_.text.strip())
                        try:
                            writer.writerow([d,article])
                        except:
                            pass
                #print(' - done with {} articles'.format(count))
def transform(x):
    final = []

    stopwords = nltk.corpus.stopwords.words('english')
    _word = TextBlob(x)
    _word = _word.words.singularize().lemmatize()
    _word = [w for w in _word if w not in stopwords]
    return ' '.join(x for x in _word)

def analyse():
    df = pd.read_csv('Articles.csv')
    df['Article'] = df['Article'].apply(lambda x: re.sub(pattern = r'[\[*\]*\"*,*.*\n*\'**\?*\:*]',repl = ' ',string = x))
    df['Article'] = df.Article.apply(lambda x:transform(x))
    df['sentiment'] = df.Article.apply(lambda x: TextBlob(x).sentiment.polarity)
    finall = {}
    for x in df.cityname.values:
        finall[x] = (df.sentiment.loc[df.cityname==x].sum())/(df[df.cityname==x].shape[0])
    return finall

In [ ]:
# run this if you want to download articles 
DownloadArticle()
analyse()

In [ ]:
#run this if you wan to use downloaded csv file
analyse()